This is a demonstration notebook to run different randomization methods on 1 dataset, and plot results.

Tissue pcHi-C dataset is used as an example.

The dataset is stored in our own format that is used in our other studies. In this notebook a large part of the code is dedicated to extract the link lists of one tissue type, and later to store the randomized links in the same format.

In [ ]:
#imports
from randomizerMain import RandLauncher
from addingNoiseOurVersion import createNoise

from copy import deepcopy
import json, csv
import time
import os

The code allows to perform all 3 types of randomization - the version that preserves both node degrees and link lengths, then the naive version of it (only preserves node degrees), and noise addition.

Finally, the randomized datasets are analyzed and plots are plotted for them.

As an example, data for chr4,8,12,16,20 is included in this repository. All chromosome data is available under "Hi-C datasets seperated by chromosome" in http://susurs.mii.lu.lv/HiCData/

In the cell below, it is possible to modify the dataset that is randomized.
To use it on your own data, you need to prepare a simple interaction list. See example in main.py

The template right now will need ~3h to run everything

In [ ]:
#Choose dataset to randomize
available_datasets = ["Tissue pcHi-C", "Tissue Hi-C", "Blood pcHI-C", "Schwarzer Hi-C"]
chosenDataset = "Tissue pcHi-C" #Choose one of the above
NUMBER_OF_REPEATS = 2 #if 2 is selected, a randomization with each set of parameters will be performed twice. 
#Expected running time: 3 hours; set NUMBER_OF_REPEATS to 1 to cut the time 2 times

dirs_with_data_files = {
    "Tissue pcHi-C": "../../sampleData/Tissue pcHiC",
    "Tissue Hi-C": "../../sampleData/Tissue HiC",
    "Blood pcHi-C": "../../sampleData/Blood pcHiC",
    "Schwarzer Hi-C": "../../sampleData/Schwarzer HiC",
}


templ = {
    "DS": chosenDataset, #short name for the dataset
    "dir": dirs_with_data_files[chosenDataset], # dir is a path to a directory that contains files with data files for chromosomes
    "number of iters": NUMBER_OF_REPEATS, #how many times to independently repeat the randomization for each graph
    "chrs" : ["chr4", "chr8", "chr12", "chr16", "chr20"], #List of chromosomes to process. These chromosomes are included in the repository for demo purposes. Data files with all chromosomes are available on http://susurs.mii.lu.lv/HiCData/
    "randomization degree targets": [0.25, 0.5, 0.75], #the parameter q for the proportion of links to attempt to randomize
    "result directory": "./demo results", #all randomized files will be stored there

    "number of iters naive": NUMBER_OF_REPEATS, #how many times to independently repeat the naive randomization for each graph
    "randomization degree targets naive": [0.2, 0.3, 0.4, 0.5, 0.6], #the parameter q for the proportion of links to attempt to randomize
    "result directory naive": "./demo results", #all randomized files will be stored there

    "number of iters noise": NUMBER_OF_REPEATS,
    "noise levels": [0.3, 0.6, 0.9, 1.2],
    "random ligation noise level": [0.0, 0.5, 1.0], #predefined, do not modify [0, 0.5, 1.0] because plotting functions expect these proportions of noise
    "result directory noise": "./demo results", 
}

if chosenDataset=="Schwarzer Hi-C":
    templ["chrs"]=["chr4", "chr8", "chr12", "chr16"] #because chr20 is unavailable

In [ ]:
#Utility functions
lastTime = time.time()
def measureTime():
    global lastTime
    delta = time.time()-lastTime
    lastTime = time.time()
    return delta
#make a dir for a file that is about to be saved
def mkDir(fullPath):
    # Extract the directory path
    dirPath = os.path.dirname(fullPath)
    if not os.path.exists(dirPath):
        os.makedirs(dirPath)

In [ ]:
times = []
for jjj in range(templ["number of iters"]):
    #for iCh in list(range(11,23))+["X"]:
    for Q in templ["randomization degree targets"]:
        for ch in templ["chrs"]:
            print(ch)
            fn = f'{templ["dir"]}/{ch}.json'
            with open(fn, 'r') as f:
                U = json.load(f)
            newU = deepcopy(U) #links will change in this newU
            ####### This portion is to extract the interaction list for each tissue type #######
            newLinkBits = {} #keys - tuples of links iin segment indeces, values - bitmaps

            tissueBits = U["tissueData"]["tissueBits"]
            allLinks = U["chrValues"][ch]["links"]
            allSegments = U["chrValues"][ch]["segments"]
            measureTime()
            for tis,tisBit in tissueBits.items(): #iterate over available tissue types in the dataset
                tissueLinks = [link for i,link in enumerate(allLinks) if ((link[2]&tisBit)==tisBit)]
                links = [[allSegments[A][0], allSegments[B][0]] for [A,B,bit] in tissueLinks]
                links = sorted(links, key=lambda x: (x[0],x[1]))
                #links are the extracted interactions for the given tissue

                obj = {
                    "useLinkList": True,
                    "inputFn": links,
                    "outputFn": f'{templ["result directory"]}/{templ["DS"]}-demoRand-{ch}-{tis}-{str(Q)}-{jjj}.csv',
                    "q": Q,
                    "runNaiveRandomization": False,
                    "coolRelated":{},
                }
                mkDir(obj["outputFn"])


                R = RandLauncher(obj) #!!! Randomization is performed here !!!
                resFn = obj["outputFn"]
                #Randomized links are stored in the csv file resFn

                ##### Now, all code below is to save the randomized links to the same format that is used in our studies and that can be processed and analyzed further
                with open(resFn, newline='') as csvfile:
                    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
                    L = list(spamreader)
                L = [[int(A),int(B)] for [A,B] in L]

                startLocusToSegmentIndex = {allSegments[i][0]:i for i,_ in enumerate(allSegments)}
                def getSegmentIndex(startLocus):
                    return startLocusToSegmentIndex[startLocus]
                
                for link in L:
                    (A,B) = sorted((getSegmentIndex(link[0]),getSegmentIndex(link[1])))
                    if (A,B) not in newLinkBits:
                        newLinkBits[(A,B)]=0
                    newLinkBits[(A,B)]=(newLinkBits[(A,B)]|tisBit)
            newLinkKeys = sorted(list(newLinkBits.keys()), key=lambda x: (x[0],x[1]))
            newLinks = []
            for key in newLinkKeys:
                newLinks.append([key[0], key[1], newLinkBits[key]])
            newU["chrValues"][ch]["links"] = newLinks

            dumpedFn = f"{templ['result directory']}/randomized-{templ['DS']}-demo-{ch}-{obj['q']}-{jjj}.json"
            with open(dumpedFn, 'w') as file:
                json.dump(newU, file)
            print(dumpedFn)
            times.append([dumpedFn, measureTime()])

            with open(f"{templ['result directory']}/timesRand-{templ['DS']}.csv", 'w', newline='') as csvfile:
                spamwriter = csv.writer(csvfile, delimiter=',',
                                        quotechar='|', quoting=csv.QUOTE_MINIMAL)
                spamwriter.writerows(times)
    

        






In [ ]:
#Now, naive randomization is to be run
times = []
for jjj in range(templ["number of iters naive"]):
    #for iCh in list(range(11,23))+["X"]:
    for Q in templ["randomization degree targets naive"]:
        for ch in templ["chrs"]:
            print(ch)
            fn = f'{templ["dir"]}/{ch}.json'
            with open(fn, 'r') as f:
                U = json.load(f)
            newU = deepcopy(U) #links will change in this newU
            ####### This portion is to extract the interaction list for each tissue type #######
            newLinkBits = {} #keys - tuples of links iin segment indeces, values - bitmaps

            tissueBits = U["tissueData"]["tissueBits"]
            allLinks = U["chrValues"][ch]["links"]
            allSegments = U["chrValues"][ch]["segments"]
            measureTime()
            for tis,tisBit in tissueBits.items(): #iterate over available tissue types in the dataset
                tissueLinks = [link for i,link in enumerate(allLinks) if ((link[2]&tisBit)==tisBit)]
                links = [[allSegments[A][0], allSegments[B][0]] for [A,B,bit] in tissueLinks]
                links = sorted(links, key=lambda x: (x[0],x[1]))
                #links are the extracted interactions for the given tissue

                obj = {
                    "useLinkList": True,
                    "inputFn": links,
                    "outputFn": f'{templ["result directory"]}/{templ["DS"]}-demoNaive-{ch}-{tis}-{str(Q)}-{jjj}.csv',
                    "q": Q,
                    "runNaiveRandomization": True, #true to run naive version
                    "coolRelated":{},
                }
                mkDir(obj["outputFn"])


                R = RandLauncher(obj) #!!! Randomization is performed here !!!
                resFn = obj["outputFn"]
                #Randomized links are stored in the csv file resFn

                ##### Now, all code below is to save the randomized links to the same format that is used in our studies and that can be processed and analyzed further
                with open(resFn, newline='') as csvfile:
                    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
                    L = list(spamreader)
                L = [[int(A),int(B)] for [A,B] in L]

                startLocusToSegmentIndex = {allSegments[i][0]:i for i,_ in enumerate(allSegments)}
                def getSegmentIndex(startLocus):
                    return startLocusToSegmentIndex[startLocus]
                
                for link in L:
                    (A,B) = sorted((getSegmentIndex(link[0]),getSegmentIndex(link[1])))
                    if (A,B) not in newLinkBits:
                        newLinkBits[(A,B)]=0
                    newLinkBits[(A,B)]=(newLinkBits[(A,B)]|tisBit)
            newLinkKeys = sorted(list(newLinkBits.keys()), key=lambda x: (x[0],x[1]))
            newLinks = []
            for key in newLinkKeys:
                newLinks.append([key[0], key[1], newLinkBits[key]])
            newU["chrValues"][ch]["links"] = newLinks

            dumpedFn = f"{templ['result directory']}/naive-{templ['DS']}-demo-{ch}-{obj['q']}-{jjj}.json"
            with open(dumpedFn, 'w') as file:
                json.dump(newU, file)
            print(dumpedFn)
            times.append([dumpedFn, measureTime()])

            with open(f"{templ['result directory']}/timesNaive-{templ['DS']}.csv", 'w', newline='') as csvfile:
                spamwriter = csv.writer(csvfile, delimiter=',',
                                        quotechar='|', quoting=csv.QUOTE_MINIMAL)
                spamwriter.writerows(times)
    

In [ ]:
#Finally, noise will be added to the dataset
for ch in templ["chrs"]:
    sourceFn = f'{templ["dir"]}/{ch}.json'
    for noiseToAdd in templ["noise levels"]:
        for rln in templ["random ligation noise level"]:
            for jjj in range(templ["number of iters noise"]):
                createNoise(sourceFn, ch, noiseToAdd, rln, jjj, templ["result directory noise"], templ["DS"])
            

Finally, the created files are analyzed in order to produce box plots to summarize results.



In [ ]:
#Utility functions

def makeAdjAndBitsOfLink(links):
    # creates an adj structure for all links and creates a dictionary bitsOfLink
    # bitsOfLink[(A,B)] == the bitmap of the link
    adj = dict()
    bitsOfLink = dict()
    for link in links:
        [A, B, bitmap, *pv] = link
        if (B<=A): 
            #print ("Warning: got an unsorted Link in makeAdj method!")
            #print(link)
            tmm=A
            A=B
            B=tmm
        if A not in adj: adj[A] = set()
        adj[A].add(B)
        if B not in adj: adj[B] = set()
        adj[B].add(A)
        bitsOfLink[(A,B)] = bitmap
    return [adj, bitsOfLink]
def getC3(links):
    # finds all triangles in current chr. At least one tissue must be shared among each link of the triangle
    # result is a list of tuples (A, B, C, bit), where A, B, C are sorted asc. and where bit is the max bitmap shared among all links (AB)&(AC)&(BC)
    # only those triangles are kept that have at least one common tissue in all 3 links
    # uses links == self.links
    # links argumentthat is used here has list of links in form [A, B, bitmap] for one chromosome
    
    [adj, bitsOfLinks] = makeAdjAndBitsOfLink(links) 
    #adj[A] == set of all other vertices that are adjacent to A
    #bitsOfLink[(A,B)] == the bitmap of the link (A,B)
    cliques = set() #{(A,B,C), (), ()}
    
    # Algorithm to find C3: take vertice A. setOfBs has all its direct neighbors (B>A to get A<B<C in result and get rid of duplicates)
    # Each node in setOfBs can be on a C3. If A-B is in some C3, then there exists a C that is both adjacent to B and to A.
    # Nodes that are adjacent to A are stored in setOfBs. Nodes that are adjacent to current B are stored in setOfCs.
    # Taking intersection - if it is non-empty, for each C, A-B-C is a triangle (clique)
    for A in adj.keys():
        setOfBs = adj[A]
        setOfBs = set(el for el in setOfBs if el>A)
        for B in setOfBs:
            setOfCs = adj[B]
            setOfCs = set(el for el in setOfCs if el>B)
            
            goodCs = setOfCs.intersection(setOfBs)
            for C in goodCs:
                [A,B,C] = sorted([A,B,C])
                bits = ( bitsOfLinks[(A,B)] & bitsOfLinks[(B,C)] & bitsOfLinks[(A,C)])
                if bits>0: #why 0? if want all CL3 not looking at tissue shared, can ignore this if
                    cliques.add((A,B,C,bits))
    del adj
    del bitsOfLinks
    return cliques 

import cooler
import csv
import os
import pandas as pd

def getCool(links, ch, resolution, resultCoolFn):
    rows=[[ch,link[0],link[0]+resolution, link[1], link[1]+resolution, 10] for link in links]
    chr_rows = {}
    for row in rows:
        if row[0] not in chr_rows:
            chr_rows[row[0]] = []
        chr_rows[row[0]].append(row)
    vertices = set()
    for chr in chr_rows:
        curr_rows = chr_rows[chr]
        for row in curr_rows:
            vertices.add((row[0], int(row[1]), int(row[2])))
            vertices.add((row[0], int(row[3]), int(row[4])))
    vertices = sorted(list(vertices))
    vertice_to_id = {x: i for i, x in enumerate(vertices)}
    edges = {}
    for chr in chr_rows:
        curr_rows = chr_rows[chr]
        for row in curr_rows:
            bin1 = (row[0], int(row[1]), int(row[2]))
            bin2 = (row[0], int(row[3]), int(row[4]))
            if bin1 > bin2:
                bin1, bin2 = bin2, bin1
            if (bin1, bin2) not in edges:
                edges[(bin1, bin2)] = int(float(row[5])*10)
            else:
                edges[(bin1, bin2)] = max(
                    edges[(bin1, bin2)], int(float(row[5])*10))
    edge_list = []
    for x in edges:
        bin1 = vertice_to_id[x[0]]
        bin2 = vertice_to_id[x[1]]
        edge_list.append((bin1, bin2, edges[x]))
        # edges.add(
        #     (vertice_to_id[bin1], vertice_to_id[bin2], row[5]))
    edges = sorted(edge_list)
    bins = {
        "chrom": [],
        "start": [],
        "end": []
    }
    for v in vertices:
        bins['chrom'].append(v[0])
        bins['start'].append(v[1])
        bins['end'].append(v[2])
    bins_df = pd.DataFrame(bins)

    pixels = {
        "bin1_id": [],
        "bin2_id": [],
        "count": []
    }
    for e in edges:
        pixels['bin1_id'].append(e[0])
        pixels['bin2_id'].append(e[1])
        pixels['count'].append(e[2])
    pixels_df = pd.DataFrame(pixels)
    cooler.create_cooler(
        resultCoolFn, bins_df, pixels_df)
    c = cooler.Cooler(resultCoolFn)
    px = c.pixels()[:10]
    #print(px)

import numpy as np
import h5py
import hicrep
from hicrep.utils import readMcool
from hicrep import hicrepSCC

def doHiCRep(fcool1, fcool2, chrs):
    
    cool1, binSize1 = readMcool(fcool1, -1)
    cool2, binSize2 = readMcool(fcool2, -1)
    # binSize1 and binSize2 will be set to the bin size built in the cool file
    binSize = binSize1
    # smoothing window half-size
    h = 1

    # maximal genomic distance to include in the calculation
    dBPMax = 50000000

    # whether to perform down-sampling or not 
    # if set True, it will bootstrap the data set # with larger contact counts to
    # the same number of contacts as in the other data set; otherwise, the contact 
    # matrices will be normalized by the respective total number of contacts
    bDownSample = False

    # compute the SCC score
    # this will result in a SCC score for each chromosome available in the data set
    # listed in the same order as the chromosomes are listed in the input Cooler files
    # scc = hicrepSCC(cool1, cool2, h, dBPMax, bDownSample)

    # Optionally you can get SCC score from a subset of chromosomes
    sccSub = hicrepSCC(cool1, cool2, h, dBPMax, bDownSample, np.array(chrs, dtype=str))
    #print(sccSub)
    return sccSub

def getAvgNodeDegreeChange(linksO, linksR):
    nodeDegrees = {} #original node degrees
    for link in linksO:
        if link[0] not in nodeDegrees:
            nodeDegrees[link[0]]=0
        nodeDegrees[link[0]]+=1
        if link[1] not in nodeDegrees:
            nodeDegrees[link[1]]=0
        nodeDegrees[link[1]]+=1
    nodeDegreesR = {} #original node degrees
    for link in linksR:
        if link[0] not in nodeDegreesR:
            nodeDegreesR[link[0]]=0
        nodeDegreesR[link[0]]+=1
        if link[1] not in nodeDegreesR:
            nodeDegreesR[link[1]]=0
        nodeDegreesR[link[1]]+=1
    diff = 0
    allNodes = set(list(nodeDegrees.keys())+list(nodeDegreesR.keys()) )
    for node in allNodes:
        origDegree=0
        if node in nodeDegrees:
            origDegree = nodeDegrees[node]
        randDegree=0
        if node in nodeDegreesR:
            randDegree = nodeDegreesR[node]
        diff+=abs(randDegree-origDegree)
    return diff

In [ ]:
L = [] #results stored here
header = []

In [ ]:
from scipy.stats import ks_2samp
def processFile(randFn, templ):
    global header
    filename = os.path.basename(randFn)
    filename = filename.replace('.json', '')
    filename = filename.replace('Hi-C', 'HiC')
    parts = filename.split('-')
    info = {
        'type': parts[0],  # e.g., 'randomized'
        'DS': parts[1],  # e.g., 'tispcHiC'
        'chr': parts[3],  # e.g., 'chr2'
        'rndDegree': float(parts[4]),  # e.g., 0.25
        'iter': float(parts[5])  # e.g., 1
    }
    with open(randFn, 'r') as f:
        R = json.load(f) #randomized file
    with open(f'{templ["dir"]}/{info["chr"]}.json', 'r') as f:
        U = json.load(f) #original file
    iii=0
    ch=info["chr"]
    oallLinks = U["chrValues"][ch]["links"] # Original all links
    oallSegments = U["chrValues"][ch]["segments"] # Original all segments
    rallLinks = R["chrValues"][ch]["links"]
    rallSegments = R["chrValues"][ch]["segments"]
    if "tissueBits" not in R: 
        tissueBits = R["tissueData"]["tissueBits"]
    else: 
        tissueBits = R["tissueBits"]
    for tis,tisBit in tissueBits.items():
        rtissueLinks = [link for i,link in enumerate(rallLinks) if ((link[2]&tisBit)==tisBit)]
        otissueLinks = [link for i,link in enumerate(oallLinks) if ((link[2]&tisBit)==tisBit)]

        rSegments = set([A for [A,B,*pv] in rtissueLinks]+[B for [A,B,*pv] in rtissueLinks])
        oSegments = set([A for [A,B,*pv] in otissueLinks]+[B for [A,B,*pv] in otissueLinks])

        #commonLinks = set([sorted((A,B)) for [A,B,*pv] in rtissueLinks]).intersection(set([sorted((A,B)) for [A,B,*pv] in otissueLinks]))
        a=set([(tuple(sorted((A,B)))) for [A,B,*pv] in otissueLinks])
        b=set([(tuple(sorted((A,B)))) for [A,B,*pv] in rtissueLinks])
        commonLinks =a.intersection(b)

        rlLengths = [abs(rallSegments[B][1]-rallSegments[A][1]) for [A,B,*pvs] in rtissueLinks] 
        olLengths = [abs(oallSegments[B][1]-oallSegments[A][1]) for [A,B,*pvs] in otissueLinks] 

        actualOLinks = [[oallSegments[A][0],oallSegments[B][0]] for [A,B,*pvs] in otissueLinks]
        actualRLinks = [[rallSegments[A][0],rallSegments[B][0]] for [A,B,*pvs] in rtissueLinks]
        print("start mk cools")
        #make sure that all segmebnts in O are also in R.
        #add some extra links
        segsInR = [A for [A,B] in actualRLinks] + [B for [A,B] in actualRLinks]
        segsInR = set(segsInR)
        segsInO = [A for [A,B] in actualOLinks] + [B for [A,B] in actualOLinks]
        segsInO = set(segsInO)
        diff = segsInO.difference(segsInR)
        for seg in diff:
            actualRLinks.append([seg,seg])

        getCool(actualOLinks, ch, 5000, "tmpCoolO.cool")
        getCool(actualRLinks, ch, 5000, "tmpCoolR.cool")
        print("made cools")
        hicreprez = doHiCRep("tmpCoolO.cool", "tmpCoolR.cool", [ch])
        print(f"{hicreprez=}")
        hicrepScore = hicreprez[0]
        

        rC3 = getC3(rtissueLinks)
        oC3 = getC3(otissueLinks)

        rez = ks_2samp(olLengths, rlLengths)
        stat = rez.statistic
        pvalue=rez.pvalue

        avgNodeDegreeChange = getAvgNodeDegreeChange(otissueLinks, rtissueLinks)

        newRow = [
            info["type"],
            info["DS"],
            info["chr"],
            info["rndDegree"],
            info["iter"],
            tis,
            len(oSegments),
            len(rSegments),
            len(otissueLinks),
            len(rtissueLinks),
            len(commonLinks),
            1-len(commonLinks)/len(rtissueLinks),
            len(oC3),
            len(rC3),
            stat,
            pvalue,
            hicrepScore,
            avgNodeDegreeChange,
        ]
        header = [
            "method", "DS", "chr", "rndDegree asked", "iter", "tis", 
            "original segments", "rnd segments", "original links", "rnd links",
            "common links", "real rnd degree",
            "original C3", "rnd C3", 
            "KS-stat","KS-pvalue",
            "HiCRep",
            "total node degree change",
        ]
        iii=0
        L.append(newRow)

In [ ]:
def store(L, header):
    with open(f'combined-results-rnd-analysis-{templ["DS"]}.csv', 'w', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerow(header)
        spamwriter.writerows(L)

In [ ]:
L = []
header = []
counter = 0
# Specify the directory path
allPaths = set([templ["result directory"], templ["result directory naive"], templ["result directory noise"]])
for directory_path in allPaths:
    # Get a list of files in the directory, sorted by file name
    sorted_files = sorted(os.listdir(directory_path))
    # Iterate over each file in the sorted list
    for file_name in sorted_files:
        # Construct the full file path
        file_path = os.path.join(directory_path, file_name)
        # Check if it's a file and not a directory (optional)
        if os.path.isfile(file_path):
            if ".csv" in file_name or ".json" not in file_name:
                continue  #only process json files
            print(file_path)
            processFile(file_path, templ)
            counter+=1
            if L: print(L[-1])
        if ((counter%10)==0): 
            store(L, header)
    store(L,header)

In [ ]:
xvalues = [0.2, 0.3, 0.4, 0.5, 0.6]
delta = 0.05
def find_closest_and_check(value, candidates=xvalues, delta=delta):
    # Find the closest value in candidates to the given value
    closest = min(candidates, key=lambda x: abs(x - value))
    # Check if the closest value is within delta of the given value
    if abs(closest - value) <= delta:
        return closest
    else:
        return -1
    
import pandas as pd

combined_df = pd.read_csv(f'combined-results-rnd-analysis-{templ["DS"]}.csv')
finalFn = f'combined-plottable-results-rnd-analysis-{templ["DS"]}.csv'
combined_df['boxSet'] = combined_df.apply(
    lambda row: 
        "Randomization preserving node degrees and link lengths" if row['method']=="randomized" else(
            "Randomization preserving node degrees" if row['method'] == "naive"
                else (
                    "100% random ligation noise" if row['iter'] == 1.0
                    else (
                        "50/50 genomic distance effect noise and random ligation noise" if row['iter'] == 0.5
                        else (
                            "100% genomic distance effect noise" if row['iter'] == 0.0
                            else f"noised {row['iter']} rln"  # Fallback case if none of the above conditions are met
                        )
                    )
                )
            ), axis=1)

combined_df['xvalue'] = combined_df['real rnd degree'].apply(find_closest_and_check)

combined_df = combined_df[combined_df['xvalue'] != -1]

combined_df['relative node degree change'] = combined_df['total node degree change']/combined_df['original segments']
combined_df['relative C3 change'] = np.where(combined_df['original C3'] == 0, 1, combined_df['rnd C3']/combined_df['original C3'])




# Save the combined DataFrame to a new file
combined_df.to_csv(finalFn, index=False)

Finally, plotting the results

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

FONT_SIZE=30

def makePlotFromTemplate(templ, ax):
    df1 = pd.read_csv(templ["fn"])
    # Define a grayscale color map for differentiation
    color_map = {iter_val: plt.cm.Greys(np.linspace(0.3, 0.8, len(df1['boxSet'].unique()))[i]) for i, iter_val in enumerate(sorted(df1['boxSet'].unique()))}

    # Initialize lists for plotting
    data_to_plot = []
    positions = []
    colors = []
    labels = []
    line_data = {}
    dash_styles = ['-', '--', '-.', ':', '-']  # Define dash styles for different iter_val
    dash_styles = [
        (5, 5),  # Dashed
        (1, 3, 1, 3, 1, 3, 6, 3),  # Three dots and a dash
        (2, 8),  # Dotted
        (4, 2, 1, 2),  # Dash-dot
        # (8, 4, 2, 4),  # Long dash, short dash
        (),  # Solid
        
    ]

    startYCoordinate=0
    if "startYCoordinate" in templ:
        startYCoordinate = templ["startYCoordinate"]

    # Adding an artificial box at the beginning for (0,0)
    data_to_plot.append([startYCoordinate])  # Artificial data
    positions.append(0)  # Position for the artificial box
    colors.append('white')  # Color for the artificial box
    labels.append('0')  # Label for the artificial box

    for iter_val in sorted(df1['boxSet'].unique()):
        df_iter = df1[df1['boxSet'] == iter_val]
        rnd_degree_groups = df_iter.groupby('xvalue')['real rnd degree'].median().sort_values()
        
        # For connecting medians, start from (0,0)
        x_positions = [0]
        y_medians = [startYCoordinate]
        
        for rnd_degree, median_val in rnd_degree_groups.items():
            group_data = df_iter[df_iter['xvalue'] == rnd_degree][templ["columnToPlot"]].values
            if iter_val in templ["removeXPoints"]:
                if rnd_degree in templ["removeXPoints"][iter_val]:
                    continue
            if len(group_data) > 0:
                data_to_plot.append(group_data)
                median_position = median_val  # Adjusted to use median_val directly
                positions.append(median_position)
                colors.append(color_map[iter_val])
                labels.append(f"{median_val:.2f}")
                
                # For lines
                x_positions.append(median_position)
                y_medians.append(np.median(group_data))
        
        # Store the line data
        line_data[iter_val] = (x_positions, y_medians)

    

    # Customizing outlier properties with smaller dots and median line properties
    flierprops = dict(marker='o', markerfacecolor='black', markersize=0.5, linestyle='none')
    medianprops = dict(color='white', linewidth=1)  # Setting medians to white
    showfliersOrNo = True
    if "showFliers" in templ:
        showfliersOrNo = templ["showFliers"]
    bp = ax.boxplot(data_to_plot, positions=positions, patch_artist=True, notch=True, widths=0.03, flierprops=flierprops, medianprops=medianprops, showfliers=showfliersOrNo)

    # Apply colors to each box and adjust properties as necessary
    for box, patch_color in zip(bp['boxes'], colors):
        box.set_facecolor(patch_color)
        box.set_edgecolor(patch_color)  # Set edge color to match the face color

    # Apply colors to each box for both face and edge, and also match whiskers and caps
    for i, box in enumerate(bp['boxes']):
        patch_color = colors[i]
        box.set_facecolor(patch_color)
        box.set_edgecolor(patch_color)  # Set edge color to match the face color
        # Set whiskers color
        bp['whiskers'][i*2].set_color(patch_color)
        bp['whiskers'][i*2 + 1].set_color(patch_color)
        # Set caps color
        bp['caps'][i*2].set_color(patch_color)
        bp['caps'][i*2 + 1].set_color(patch_color)

    for i, (iter_val, (x_pos, y_vals)) in enumerate(line_data.items()):
        # Retrieve the dash style
        dash_style = dash_styles[i % len(dash_styles)]
        # Plot the line without specifying dash style
        line, = ax.plot(x_pos, y_vals, color=color_map[iter_val], label=f'Iter {iter_val}')
        # Apply dash style
        if isinstance(dash_style, tuple):
            line.set_dashes(dash_style)  # For custom dash patterns
        elif isinstance(dash_style, str):
            line.set_linestyle(dash_style)  # For predefined dash styles as strings

    # Customizing the axes
    ax.set_xticks(np.arange(0, max(positions) + 0.1, 0.1))  # Adjust based on your actual data range
    ax.set_xticklabels([f"{x:.1f}" for x in np.arange(0, max(positions) + 0.1, 0.1)], rotation=45, ha='right')
    ax.tick_params(axis='both', labelsize=18)  # Adjusts both x and y tick label sizes

    if positions:
        min_pos = min(positions)
        max_pos = max(positions)
        buffer = (max_pos - min_pos) * 0.05  # Add 5% buffer on each side
        ax.set_xlim(min_pos - buffer, max_pos + buffer)

    ax.set_ylabel(templ["ylabel"], fontsize=18)
    ax.set_xlabel(templ["xlabel"], fontsize=18)
    ax.set_title(templ["title"], fontsize=20)

    ax.grid(True, axis='y', linestyle='--', linewidth=0.7)

    legend_elements=[]
    for i, iter_val in enumerate(sorted(df1['boxSet'].unique())):
        dash_style = dash_styles[i % len(dash_styles)]
        line = plt.Line2D([0], [0], color=color_map[iter_val], lw=2, label=df1[df1['boxSet'] == iter_val].iloc[0]['boxSet'])
        if isinstance(dash_style, tuple):
            line.set_dashes(dash_style)  # For custom dash patterns
        else:
            line.set_linestyle(dash_style)  # For predefined styles
        legend_elements.append(line)
    # # Update legend to include dash styles
    # legend_elements = [
    #     plt.Line2D([0], [0], color=color_map[iter_val], lw=2, linestyle=dash_styles[i % len(dash_styles)], label=df1[df1['boxSet'] == iter_val].iloc[0]['boxSet'])
    #     for i, iter_val in enumerate(sorted(df1['boxSet'].unique()))
    # ]
    # showLegend=True
    # if "showLegend" in templ :
    #     showLegend = templ["showLegend"]
    # if showLegend:
    #     ax.legend(handles=legend_elements, title="Randomization technique")
    ax.legend(handles=legend_elements, title="Randomization technique", fontsize=12, handlelength=4)

    # legend_elements = [plt.Line2D([0], [0], color=color_map[iter_val], lw=2, linestyle=dash_styles[i % len(dash_styles)], label=f'Iter {iter_val}') for i, iter_val in enumerate(sorted(df1['iter'].unique()))]
    # ax.legend(handles=legend_elements, title="Iter Values and Styles")

    plt.tight_layout()

In [ ]:

sf=True
templPlotTissuePCHiC_KS = {
    "columnToPlot": 'KS-stat',
    "ylabel": "KS test statistic",
    "xlabel": "Real randomization degree",
    "title": f"2-sample Kolmogorov-Smirnov test statistic to compare\n link length distribution of randomized graphs",
    "removeXPoints":    {
                            "Randomization preserving node degrees": set([0.4, 0.5, 0.6, 0.7]),
                            #"100% random ligation noise": set([0.4]), 
                        },
    "fn": f'combined-plottable-results-rnd-analysis-{templ["DS"]}.csv',
    "showFliers": sf,
}
templPlotTissuePCHiC_ND = {
    "columnToPlot": 'relative node degree change',
    "ylabel": "Relative node degree change",
    "xlabel": "Real randomization degree",
    "title": f"Average node degree change in randomized graphs\n",

    "removeXPoints":    {},
    "fn": f'combined-plottable-results-rnd-analysis-{templ["DS"]}.csv',
    "showFliers": sf,
}

templPlotTissuePCHiC_HR = {
    "columnToPlot": 'HiCRep',
    "ylabel": "HiCRep reproducibility statistic",
    "xlabel": "Real randomization degree",
    "title": f"HiCRep reproducibility statistic change in randomized graphs",
    "removeXPoints":    {},
    "fn": f'combined-plottable-results-rnd-analysis-{templ["DS"]}.csv',
    "showFliers": sf,
    "startYCoordinate": 1.0,

}
sf=False
templPlotTissuePCHiC_C3 = {
    "columnToPlot": 'relative C3 change',
    "ylabel": "Relative C3 change",
    "xlabel": "Real randomization degree",
    "title": "Relative C3 change in randomized graphs",
    "removeXPoints":    {},
    "fn": f'combined-plottable-results-rnd-analysis-{templ["DS"]}.csv',
    "showFliers": sf,
    "startYCoordinate": 1.0,

}

In [ ]:
#fig, ax = plt.subplots(figsize=(12, 6))
fig, axs = plt.subplots(2, 2, figsize=(18,14))

makePlotFromTemplate(templPlotTissuePCHiC_KS, axs[0,0])
makePlotFromTemplate(templPlotTissuePCHiC_ND, axs[0,1])
makePlotFromTemplate(templPlotTissuePCHiC_HR, axs[1,0])
makePlotFromTemplate(templPlotTissuePCHiC_C3, axs[1,1])

#axs[0,0].legend().remove()
# axs[0,1].legend().remove()
# axs[1,0].legend().remove()
# axs[1,1].legend().remove()

# Add a supertitle to the figure
fig.suptitle(f'{templ["DS"]}', fontsize=FONT_SIZE)
# Adjust layout to make room for the supertitle
plt.tight_layout(rect=[0, 0.03, 1, 0.97])

# Now, save the figure to an EPS file with a specific DPI
# fig.savefig(f'{templ["DS"]}-4images.eps', format='eps', dpi=600)
fig.savefig(f'{templ["DS"]}-4images.png', dpi=600)


plt.show()